In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

C:\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [2]:
movies_df = pd.read_csv('ml-1m/movies.dat', sep="::", header=None)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
movies_df.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings_df = pd.read_csv('ml-1m/ratings.dat', sep="::", header=None)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [5]:
ratings_df.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
movies_df.columns = ["Movie_id", "Movie_title", "Genres"]
ratings_df.columns = ["User_id", "Movie_id","Rating", "Timestamp"]

In [7]:
movies_df.head()

,Movie_id,Movie_title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
ratings_df.head()

,User_id,Movie_id,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [9]:
movies_df['List_Index'] = movies_df.index

In [10]:
movies_df.head()

,Movie_id,Movie_title,Genres,List_Index
0,1,Toy Story (1995),Animation|Children's|Comedy,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1
2,3,Grumpier Old Men (1995),Comedy|Romance,2
3,4,Waiting to Exhale (1995),Comedy|Drama,3
4,5,Father of the Bride Part II (1995),Comedy,4


In [11]:
merged_df = movies_df.merge(ratings_df, on="Movie_id")

In [12]:
merged_df = merged_df.drop('Timestamp', axis=1).drop('Genres', axis=1).drop('Movie_title',axis=1)

In [13]:
merged_df.head()

,Movie_id,List_Index,User_id,Rating
0,1,0,1,5
1,1,0,6,4
2,1,0,8,4
3,1,0,9,5
4,1,0,10,5


In [17]:
userGroup = merged_df.groupby("User_id")
userGroup.first().tail()

,Movie_id,List_Index,Rating
User_id,,,
6036,4,3,2
6037,17,16,4
6038,232,229,4
6039,48,47,3
6040,1,0,3
